In [22]:
import csv
import openai
import os
from openai import OpenAI
import os
import time
import random
import json
import logging
from datetime import datetime
import tqdm
import pickle
import pandas as pd
import math

#replace "API-KEY" with personalized API-KEY
client = OpenAI(api_key=os.environ['CHEN_OPENAI_API_KEY'])

In [5]:
# The columns we expect in our final sheet
OUTPUT_COLUMNS = [
    "Title of the Study",
    "Mediation Method Used",
    "Randomized Exposure",
    "Causal Mediation",
    "Examined Mediator-Outcome Linearity",
    "Examined Exposure-Mediator Interaction",
    "Covariates in Exposure-Mediator Model",
    "Covariates in Exposure-Outcome Model",
    "Covariates in Mediator-Outcome Model",
    "Control for Baseline Mediator",
    "Control for Baseline Outcome",
    "Temporal Ordering Exposure Before Mediator",
    "Temporal Ordering Mediator Before Outcome",
    "Discussed Mediator Assumptions",
    "Sensitivity Analysis to Assumption",
    "Control for Other Post-Exposure Variables",
]

In [7]:
Adam_Ian_All = pd.read_excel("/Users/yiquntchen/Dropbox/causaljudge-data/Mediation-cathy-study/Adam_Ian/Adam_Ian_All.xlsx",
                                engine=None)
Adam_Ian_All = Adam_Ian_All[~pd.isna(Adam_Ian_All['PMID#'])]

In [8]:
Adam_Ian_All

,Unnamed: 0,PMID#,Authors,Title
0,1.0,22998852.0,"Ali A, Ambler G, Strydom A, Rai D, Cooper C, M...",The relationship between happiness and intelli...
1,2.0,23025248.0,Sikkema et al.,Reductions in Traumatic Stress following a Cop...
2,3.0,23527848.0,"Guimond S, Crisp RJ, De Oliveira P, Kamiejski ...","Diversity policy, social dominance, and interg..."
3,4.0,23815233.0,"Delton AW, Nemirow J, Robertson TE, Cimino A, ...",Merely Opting Out of a Public Good Is Moralize...
4,5.0,24022649.0,"Weisleder A, Fernald A",Talking to children matters: early language ex...
5,6.0,25031222.0,"Freeman D, Dunn G, Murray RM, et al.",How Cannabis Causes Paranoia: Using the Intrav...
6,7.0,25039961.0,"Pickles A, Harris V, Green J, Aldred C, McCona...",Treatment mechanism in the MRC preschool autis...
7,8.0,25071670.0,Real et al,Well-being in amyotrophic lateral sclerosis: a...
8,9.0,25192244.0,"Lin T, Vaisvaser S, Fruchter E, Admon R, Wald ...",A neurobehavioral account for individual diffe...
9,10.0,25451409.0,Garthus-Niegel et al.,Maintaining factors of posttraumatic stress sy...


In [31]:
# Open and read the jsonl file
with open('/Users/yiquntchen/Dropbox/causaljudge-data/PMID_all_text.jsonl', 'r') as f:
    PMID_all_text = [json.loads(line) for line in f]
result = {}

In [42]:
def get_response_sync(client, prompt, model, temp=0):
    """Synchronously get response from OpenAI API"""
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt},
        ],
        response_format={"type": "json_object"},
        temperature=temp,
    )
    return completion.choices[0].message.content

In [44]:
result = {}
#o4-mini gpt-4.1-mini gpt-4o-mini
gpt_model = #"o4-mini" #"gpt-4.1-mini" #"o4-mini"#"gpt-4.1"#"gpt-4o"
if gpt_model == "o4-mini":
    temp = 1.0
else:
    temp = 0.0
for entry in tqdm.tqdm(PMID_all_text):
    pmid = entry.get("pmid")
    
    # Skip if already processed
    if pmid in result:
        continue

    article_content = entry.get("text")

    # Prepare your prompt (you need to define this part)
    testing_prompt = f"""
    You are a research expert specializing in causal mediation analysis.  
    You will be given the OCR-converted text sections of a scientific article.  
    Your task is to extract structured information from this article and return it in a **valid JSON format**.  
    
    The provided PMID is: {pmid}
    The article provided is:{article_content}.
    
    The extracted information must be structured as a JSON object containing the following fields:
    
    Required Fields to Extract:
    - "pmid_number": [PMID number of the article]
    - "authors": [List or comma-separated names if available; otherwise "Not mentioned"]
    - "title": [Title of the article]
    - "journal": [Journal name, if mentioned; otherwise "Not mentioned"]
    - "data_available": [yes/no; did the authors make data available?]
    - "sample_description": [Brief sample description if available; otherwise "Not mentioned"]
    - "sample_size": [Numeric sample size if explicitly stated; otherwise "Not mentioned"]
    - "clustering": [yes/no; was clustering mentioned?]
    - "clustering_dealt_with": [yes/no; was clustering statistically handled?]
    - "exposure_s": [Exposure(s) investigated]
    - "mediator_s": [Mediator(s) studied]
    - "outcome_s": [Outcome(s) studied]
    - "exposure_randomized": [yes/no; was the exposure randomized?]
    - "name_of_mediation_method": [Name of the mediation method used; ≤10 words if missing]
    - "citation_for_mediation_method": [If a citation is given for the mediation method; otherwise "Not mentioned"]
    - "causal_mediation_yes_no": [yes/no; was a causal mediation framework used?]
    - "if_causal_which_estimand": [1=Controlled Direct/Indirect Effects; 2=Natural Direct/Indirect Effects; 3=Stochastic/Randomized Interventional Effects; "Not mentioned" if unclear]
    - "effects_of_interest_nie_nde_etc": [Mentioned effects like NIE, NDE, etc.; otherwise "Not mentioned"]
    - "if_baron_kenny_examine_interaction": [yes/no; did they test interaction between exposure and mediator?]
    - "if_baron_kenny_examine_linearity": [yes/no; did they test linearity between mediator and outcome?]
    - "covariates_in_exposure_mediator_model": [yes/no; covariates controlled for in exposure → mediator model?]
    - "covariates_in_mediator_outcome_model": [yes/no; covariates controlled for in mediator → outcome model?]
    - "covariates_in_exposure_outcome_model": [yes/no; covariates controlled for in exposure → outcome model?]
    - "any_justification_for_covariates": [yes/no; did they justify covariate selection?]
    - "baseline_value_of_mediator_adjusted_for": [yes/no; adjusted for baseline value of mediator?]
    - "baseline_value_of_outcome_adjusted_for": [yes/no; adjusted for baseline value of outcome?]
    - "temporal_ordering_of_exposure_before_mediator_yes_no": [yes/no]
    - "temporal_ordering_of_mediator_before_outcome_yes_no": [yes/no]
    - "discussion_of_mediation_assumptions_yes_no": [yes/no; did they discuss assumptions like no unmeasured confounding?]
    - "sensitivity_analyses_to_assumptions_yes_no": [yes/no; did they conduct sensitivity analyses for assumptions?]
    - "does_model_control_for_other_post_exposure_variables_yes_no": [yes/no; did they adjust for post-exposure variables?]
    
    Format your JSON output with exactly the following *KEYS*, but for exposures/mediators/outcomes, feel free to input details.
    {{
        "pmid_number": "xxxxxx",
        "authors": "Author A, Author B, Author C",
        "title": "Study title here",
        "journal": "Journal Name",
        "data_available": "yes",
        "sample_description": "Description of study sample",
        "sample_size": "250",
        "clustering": "no",
        "clustering_dealt_with": "no",
        "exposure_s": "List or description",
        "mediator_s": "List or description",
        "outcome_s": "List or description",
        "exposure_randomized": "yes",
        "name_of_mediation_method": "Method name",
        "citation_for_mediation_method": "Author Year or Method Name",
        "causal_mediation_yes_no": "yes",
        "if_causal_which_estimand": "2",
        "effects_of_interest_nie_nde_etc": "NIE, NDE",
        "if_baron_kenny_examine_interaction": "no",
        "if_baron_kenny_examine_linearity": "yes",
        "covariates_in_exposure_mediator_model": "yes",
        "covariates_in_mediator_outcome_model": "yes",
        "covariates_in_exposure_outcome_model": "yes",
        "any_justification_for_covariates": "no",
        "baseline_value_of_mediator_adjusted_for": "no",
        "baseline_value_of_outcome_adjusted_for": "yes",
        "temporal_ordering_of_exposure_before_mediator_yes_no": "yes",
        "temporal_ordering_of_mediator_before_outcome_yes_no": "yes",
        "discussion_of_mediation_assumptions_yes_no": "yes",
        "sensitivity_analyses_to_assumptions_yes_no": "no",
        "does_model_control_for_other_post_exposure_variables_yes_no": "no"
      }}
    """
    try:
        completion = get_response_sync(client, testing_prompt, gpt_model, temp=temp)
        result[pmid] = completion
    except Exception as e:
        print(f"Error processing PMID {pmid}: {e}")

    # Optional: Save progress after every step
    with open(f'/Users/yiquntchen/Dropbox/causaljudge-data/{gpt_model}_result_partial.json', 'w') as f_out:
        json.dump(result, f_out)
with open(f'/Users/yiquntchen/Dropbox/causaljudge-data/{gpt_model}_result_final.json', 'w') as f_out:
    json.dump(result, f_out)

100%|█████████████████████████████████████████| 180/180 [56:12<00:00, 18.74s/it]


In [27]:
completion_1 = get_response_sync(client, testing_prompt, "gpt-4o")

In [28]:
completion_1

'{\n    "pmid_number": "24813803",\n    "authors": "Dustin E. Sarver, Michael R. McCart, Ashli J. Sheidow, Elizabeth J. Letourneauf",\n    "title": "ADHD and risky sexual behavior in adolescents: Conduct problems and substance use as mediators of risk",\n    "journal": "Journal of Child Psychology and Psychiatry",\n    "data_available": "no",\n    "sample_description": "Ethnically diverse cross-sectional sample of adolescents involved in the juvenile justice system",\n    "sample_size": "115",\n    "clustering": "no",\n    "clustering_dealt_with": "no",\n    "exposure_s": "ADHD symptoms",\n    "mediator_s": "Conduct problems, problematic alcohol use, problematic marijuana use",\n    "outcome_s": "Risky sexual behavior",\n    "exposure_randomized": "no",\n    "name_of_mediation_method": "Bootstrapped mediation models",\n    "citation_for_mediation_method": "Hayes & Scharkow, 2013; Shrout & Bolger, 2002",\n    "causal_mediation_yes_no": "no",\n    "if_causal_which_estimand": "Not mention

In [ ]:

        #Call GPT API
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are a research assistant analyzing mediation studies."},
                {"role": "user", "content": prompt}
            ],
            temperature=0,
            max_tokens=500
        )
        response_text = completion.choices[0].message.content.strip()
        response_dict = {col: "" for col in OUTPUT_COLUMNS}
        
        lines = response_text.split("\n")
        
        for line in lines:
            line = line.strip()
            if not line or ":" not in line:
                continue
            
            if line.startswith("- "):
                line = line[2:].strip()

            parts = line.split(":", 1)
            if len(parts) < 2:
                continue
            
            key_raw = parts[0].strip()
            val_raw = parts[1].strip()
            
            if key_raw in response_dict:
                response_dict[key_raw] = val_raw

        rows_to_write.append(response_dict)

#Write results to a new CSV
output_filename = "extracted_Trang.csv"
with open(output_filename, mode="w", newline='', encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=OUTPUT_COLUMNS)
    writer.writeheader()
    for row_data in rows_to_write:
        writer.writerow(row_data)

print(f"Done! Results saved to {output_filename}. Open it in Excel to see each question as a column.")
